In [61]:
import csv
from itertools import combinations
import pandas as pd
from sklearn.model_selection import train_test_split
import evaluate
from sentence_transformers import CrossEncoder, SentenceTransformer
import numpy as np
from sklearn.cluster import AgglomerativeClustering

In [87]:
rouge = evaluate.load("rouge")

In [2]:
dev = pd.read_csv('ru-dev.csv', sep='\t', quoting=csv.QUOTE_NONE)

In [14]:
def create_dataset(dev, lang):
    to_return = {'gloss':[], 'Generated_Definition':[], 'score':[]}
    for word in dev.word.unique():
        this_word = dev[dev.word == word]
        for sense in this_word.sense_id.unique():
            this_sense = this_word[this_word.sense_id==sense]

            to_predict = list(this_sense[['gloss', 'Generated_Definition']].dropna().itertuples(index=False, name=None))
            if to_predict:
                sims = model.predict(to_predict)
                most_similar = to_predict[np.argmax(sims)]
                to_return['gloss'].append(most_similar[0])
                to_return['Generated_Definition'].append(most_similar[1])
                to_return['score'].append(1)
            
            other_sense = this_word[this_word.sense_id!=sense].Generated_Definition.dropna()
            for other, this in zip(other_sense, this_sense):
                to_return['gloss'].append(this)
                to_return['Generated_Definition'].append(other)
                to_return['score'].append(0)
    to_return = pd.DataFrame(to_return)
    to_return['lang'] = [lang for i in range(to_return.shape[0])]
    return to_return

In [13]:
dev.head()

,Unnamed: 0,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period,Real_Contexts,Generated_Definition
0,0,dev_ru_0,могильник,могильникъ,mogil'nik_UYTE5-B076I,вят. слово из Вятской губернии арх. слово из А...,"могилки, кладбище",NaN,old,old,"могилки, кладбище Что такое могильник?","кладбище, могильная камера"
1,1,dev_ru_1,могильник,могильникъ,mogil'nik_C3GhETZc5Vs,ярс. слово из Ярославской губернии крупный коч...,NaN,NaN,old,old,NaN,NaN
2,2,dev_ru_2,могильник,могильникъ,mogil'nik_KNs3eVn3pFY,арх. слово из Архангельской губернии походный ...,NaN,NaN,old,old,NaN,NaN
3,3,dev_ru_3,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,"Орёл изображает реку Халзан, его голова ― скал...",NaN,new,new,"Орёл изображает реку Халзан, его голова ― скал...",арх. слово из Архангельской губернии
4,4,dev_ru_4,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,Орел могильник,NaN,old,old,Орел могильник Что такое могильник?,арх. слово из Архангельской губернии


In [8]:
model = CrossEncoder("BAAI/bge-reranker-v2-m3")

In [15]:
ru = create_dataset(dev, 'ru')

In [16]:
ru.shape

(8369, 4)

In [20]:
ru = ru.drop_duplicates()
ru.shape

(4209, 4)

In [58]:
fin = pd.read_csv('../dev_set/mt0-xl/ax_fi_2.tsv.gz', sep='\t', quoting=csv.QUOTE_NONE, compression='gzip')
fi = create_dataset(fin, 'fi')

(3356, 3)


In [59]:
dataset = pd.concat((ru, fi))

In [45]:
train, test = train_test_split(ru, shuffle=True, stratify=ru['score'], random_state=42)

In [22]:
import torch
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from Levenshtein import distance, ratio
import swifter

In [84]:
to_predict = list(test[['gloss', 'Generated_Definition']].itertuples(index=False, name=None))
sims_test = model.predict(to_predict)
test['sims'] = sims_test
# test['lev'] = test.apply(lambda x: ratio(x.gloss, x.Generated_Definition), axis=1)
# test.to_csv('test_sims.csv')

In [81]:
train

,gloss,Generated_Definition,score,lang,sims
629,сыпаться через край;,"пересыпая, засыпая более чем столько, чтобы не...",0,ru,0.208872
1559,"свойственный бане, характерный для неё","связанный, соотносящийся по значению с существ...",1,ru,0.192625
1088,"событие, лишившее кого-то собственности или до...","перен. потера чего-либо, лишение кого-либо, че...",1,ru,0.742397
106,перен. обеспечивающий достижение цели; верный,"обладающий крепостью, способный выдерживаться ...",0,ru,0.012446
262,мужское имя,"устар. и разг. кличка, используемая для иденти...",0,ru,0.001389
...,...,...,...,...,...
1248,город Пруса,устар. и истор. род племени в северных областя...,0,ru,0.001371
1713,"явление действительности, факт, обстоятельство...","действие по значению гл. вещать; то, что имеет...",0,ru,0.065916
1253,"жарг. узнать у кого-либо что-либо, получить ка...","перен., разг. улучшить качество, уровень чего-...",1,ru,0.005224
1361,"перен. освободить место, проход куда-либо, зас...","очищая, удалять загрязнения, грязь, пыль",0,ru,0.003262


In [82]:
test

,gloss,Generated_Definition,score,lang
886,"разг. наскоро, слегка или в отдельных местах в...","поднять, помочь чему-либо помыться",1,ru
1399,"школа (вообще место, откуда разносится наука, ...","перен., разг. источник, рассадная площадь для ...",1,ru
818,"выпустить, испустить, извергнуть, источить","разг., обычно с доп. вынудять прекратиться, пр...",1,ru
1001,способ рыбалки,"устар. и рег. набор, сборище",0,ru
705,трансп. специально оборудованная на маршруте т...,"перен., разг. основание, на котором располагаю...",0,ru
...,...,...,...,...
1172,действие по глаголу проклинать. Наложить на ко...,устар. проклятье,0,ru
840,из пуха или с пухом; на пуху; валеная из звери...,разг. содержащий пух,1,ru
1067,"перен., разг. приезжать, приплывать куда-либо ...","идя, покидать [ I ] какое-либо место, то есть ...",0,ru
1669,имеющий бороду или сильно заросший бородой (о ...,"перен., разг. имеющий борода",1,ru


In [24]:
def predict_sims(ds):
    to_predict = list(ds[['gloss', 'Generated_Definition']].itertuples(index=False, name=None))
    sims = model.predict(to_predict)
    ds['sims'] = sims
    return ds

In [102]:
ru

,gloss,Generated_Definition,score,lang
0,вят. слово из Вятской губернии арх. слово из А...,"охотник, убийца",0,ru
1,ярс. слово из Ярославской губернии крупный коч...,кладбище,0,ru
2,арх. слово из Архангельской губернии походный ...,кладбище,0,ru
3,орнитол. хищная птица рода крупных степных орлов,кладбище,0,ru
4,орнитол. хищная птица рода крупных степных орлов,курганное захоронение,0,ru
...,...,...,...,...
2021,"перен. разг. страстное увлечение, возбуждение;...","сильное, быстрое повышенное воспаление",1,ru
2022,"перен. разг. страстное увлечение, возбуждение;...","сильное, неприятное чувство",1,ru
2023,"перен. разг. страстное увлечение, возбуждение;...","творческая горячность, страстность",1,ru
2024,"перен. напряженная работа, требующая немедленн...","сильное эмоциональное состояние, часто внезапное",1,ru


In [25]:
# dev = pd.read_csv('ru-dev-aya.csv', sep='\t', quoting=csv.QUOTE_NONE)
# ru = create_dataset(dev, 'ru')
# train, test = train_test_split(ru, shuffle=True, stratify=ru['score'], random_state=42)
train = predict_sims(train)
test = predict_sims(test)
clf = tree.DecisionTreeClassifier()

clf = clf.fit(train[['sims']], train['score'])
predictions = clf.predict(test[[ 'sims']])
print(classification_report(test['score'], predictions))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       832
           1       0.63      0.63      0.63       221

    accuracy                           0.84      1053
   macro avg       0.77      0.77      0.77      1053
weighted avg       0.84      0.84      0.84      1053



In [28]:
rouge = evaluate.load("rouge")

In [38]:
def calc_rouge(ds):
    ds['rouge'] = ds.apply(lambda x: rouge.compute(
    predictions=[x.gloss+'\n'], references=[x.Generated_Definition+'\n'], tokenizer=lambda y: y.split()
), axis=1)
    ds['rouge1'] = [x['rouge1'] if isinstance(x, dict) else 0 for x in ds['rouge']]
    ds['rouge2'] = [x['rouge2'] if isinstance(x, dict) else 0 for x in ds['rouge']]
    ds['rougeL'] = [x['rougeL'] if isinstance(x, dict) else 0 for x in ds['rouge']]
    ds['rougeLsum'] = [x['rougeLsum'] if isinstance(x, dict) else 0 for x in ds['rouge']]
    return ds

In [26]:
clf =LogisticRegression()

clf = clf.fit(train[['sims']], train['score'])
predictions = clf.predict(test[[ 'sims']])
print(classification_report(test['score'], predictions))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93       832
           1       0.91      0.50      0.64       221

    accuracy                           0.88      1053
   macro avg       0.89      0.74      0.79      1053
weighted avg       0.89      0.88      0.87      1053



In [47]:
train = predict_sims(train)
train = calc_rouge(train)
test = calc_rouge(test)

In [48]:
train.head()

,gloss,Generated_Definition,score,lang,sims,rouge,rouge1,rouge2,rougeL,rougeLsum
8090,Unnamed: 0,свойство по значению прилагатника гладкий; лаб...,0,ru,0.000032,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.0,0.0,0.0,0.0
5101,date,"действие по значению гл. прогонять; обыск, вые...",0,ru,0.000024,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.0,0.0,0.0,0.0
7125,gloss,"перен., бран. скверный, низменный, ничтожный ч...",0,ru,0.018614,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.0,0.0,0.0,0.0
5787,orth,"разг. удалять откуда-либо, снять с чего-либо",0,ru,0.025723,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.0,0.0,0.0,0.0
6484,orth,"часть автомобиля, предназначенная для хранения...",0,ru,0.002535,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.0,0.0,0.0,0.0


In [49]:
test.head()

,gloss,Generated_Definition,score,lang,rouge,rouge1,rouge2,rougeL,rougeLsum
6479,Real_Contexts,"базовая, стояночная площадь для какого-либо ав...",0,ru,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
7515,usage_id,"разг., устар. бедная женщина",0,ru,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
7412,Unnamed: 0,"перен., разг. жить, существовать в процессе пр...",0,ru,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
7736,indices_target_token,"период в истории человечества, определённый ка...",0,ru,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
5050,действие по значению глагола прогонять,действие по значению гл. гонять; бег,1,ru,"{'rouge1': 0.5454545454545454, 'rouge2': 0.444...",0.545455,0.444444,0.545455,0.545455


In [60]:
clf =LogisticRegression()

clf = clf.fit(train[['sims', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum']], train['score'])
#test = predict_sims(test)

predictions = clf.predict(test[[ 'sims', 'rouge1', 'rouge2', 'rougeL', 'rougeLsum']])
print(classification_report(test['score'], predictions))

              precision    recall  f1-score   support

           0       0.89      0.99      0.94       832
           1       0.92      0.56      0.69       221

    accuracy                           0.90      1053
   macro avg       0.91      0.77      0.82      1053
weighted avg       0.90      0.90      0.89      1053



In [63]:
encoder =SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

In [65]:
train.columns

Index(['gloss', 'Generated_Definition', 'score', 'lang', 'sims', 'rouge',
       'rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
      dtype='object')

In [67]:
dev_new = dev[dev.period=='new']
dev_new.head()

,Unnamed: 0,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period,Real_Contexts,Generated_Definition
3,3,dev_ru_3,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,"Орёл изображает реку Халзан, его голова ― скал...",NaN,new,new,"Орёл изображает реку Халзан, его голова ― скал...",арх. слово из Архангельской губернии
7,7,dev_ru_7,могильник,могильникъ,mogil'nik_FciH4mIkAEU,археол. в древности — комплекс многих погребен...,В экспозиции представлен инвентарь из реликтов...,NaN,new,new,В экспозиции представлен инвентарь из реликтов...,арх. слово из Архангельской губернии
8,8,dev_ru_8,могильник,могильникъ,mogil'nik_FciH4mIkAEU,археол. в древности — комплекс многих погребен...,Близ Барбашиной поляны отрыт мордовский могиль...,NaN,new,new,Близ Барбашиной поляны отрыт мордовский могиль...,арх. слово из Архангельской губернии
9,9,dev_ru_9,могильник,могильникъ,mogil'nik_W-DBS9zwZiI,спец. место захоронения радиоактивных отходов;...,О строительстве могильника для захоронения рад...,NaN,new,new,О строительстве могильника для захоронения рад...,арх. слово из Архангельской губернии
10,10,dev_ru_10,могильник,могильникъ,mogil'nik_W-DBS9zwZiI,спец. место захоронения радиоактивных отходов;...,"Якобы тут, возле Черниховки, находится радиоак...",NaN,new,new,"Якобы тут, возле Черниховки, находится радиоак...","перен., книжн. кладбище, могильная местность"


In [92]:
from sklearn.metrics import adjusted_rand_score
from collections import Counter
from leven import levenshtein

In [104]:
def find_merges(df):
    targ_words = df.word.unique()

    # Finding definitions similar to the most frequent one (paraphrases) for each targ_word:
    mappings = {targ_word: {} for targ_word in targ_words}

    for targ_word in targ_words:
        definitions = Counter(df[df.word == targ_word].Generated_Definition.dropna()).most_common()
        if definitions:
        # definitions = sorted(definitions)  # can be commented to out to start from most frequent
            sim_cache = {}
            def2compare = None
            for nr, source in enumerate(definitions):
                cand = source[0]
                if len(cand.split()) >= 4 and cand not in mappings[targ_word]:
                    def2compare = cand
                    mapped = 0
                    for definition in definitions:
                        definition_text = definition[0]
                        if definition_text != def2compare and \
                                definition_text not in mappings[targ_word]:
                            if (definition_text, def2compare) in sim_cache:
                                distance = sim_cache[(definition_text, def2compare)]
                            else:
                                distance = levenshtein(definition_text, def2compare)
                                sim_cache[(definition_text, def2compare)] = distance
                                sim_cache[(def2compare, definition_text)] = distance
                            if distance < 10:
                                mappings[targ_word][definition_text] = def2compare
                                mapped += 1

                if def2compare:
                    break

    return mappings, targ_words

In [105]:
mappings, targ_words = find_merges(train_new)

In [106]:
mappings

{'могильник': {},
 'монитор': {},
 'мороз': {'устар. холодная погодa': 'устар. холодное время года'},
 'мундштук': {'часть трубки, соединяющая рот с остальной частью папиросы': 'часть трубки, соединяющая рот с остальной частью трубки'},
 'муж': {},
 'мужчина': {'лингв., разг. человек, принадлежающий мужчине [ 1 ]': 'лингв., устар. человек, принадлежающий к мужчине [ 1 ]'},
 'мурман': {},
 'мясо': {},
 'нос': {},
 'нагулять': {},
 'надёжный': {'обладающий высоким уровнем надежности, стойкости': 'обладающий высоким уровнем надежности, стабильности'},
 'надой': {},
 'назначить': {},
 'наиграть': {},
 'наносить': {},
 'наново': {},
 'наносный': {},
 'нараспашку': {},
 'номер': {},
 'нора': {},
 'нота': {},
 'набрать': {},
 'наверх': {},
 'находить': {},
 'натирать': {},
 'натурный': {'настоящий, истиный, действующий на деле': 'настоящий, живой, действующий на деле'},
 'нейтральный': {},
 'неподвижный': {},
 'нива': {},
 'низменный': {},
 'ник': {},
 'обводить': {'обрисовываться, рисовать п

In [107]:
targ_words

array(['могильник', 'монитор', 'мороз', 'мундштук', 'муж', 'мужчина',
       'мурман', 'мясо', 'нос', 'нагулять', 'надёжный', 'надой',
       'назначить', 'наиграть', 'наносить', 'наново', 'наносный',
       'нараспашку', 'номер', 'нора', 'нота', 'набрать', 'наверх',
       'находить', 'натирать', 'натурный', 'нейтральный', 'неподвижный',
       'нива', 'низменный', 'ник', 'обводить', 'облако', 'областной',
       'обмывать', 'обрывать', 'обух', 'отступать', 'отдать', 'отрубить',
       'отчаянный', 'оговорить', 'округлить', 'общий', 'опадать',
       'оптика', 'опутать', 'лента', 'летун', 'лещ', 'лизать', 'линия',
       'лист', 'литовка', 'логовище', 'ложе', 'лох', 'луковица', 'падла',
       'паз', 'папка', 'парафраз', 'парк', 'паровоз', 'партия', 'пачка',
       'перевёртыш', 'передок', 'передовой', 'пересыпать', 'перетереть',
       'перо', 'петрушка', 'пионер', 'платформа', 'плотный', 'поболтать',
       'побывать', 'погасить', 'пул', 'пульс', 'пустить', 'путь',
       'пуховый',

In [108]:
new_defs = []
for word in train_new.word.unique():
    word_map = mappings[word]
    this_word = train_new[train_new.word==word]
    for defi in this_word.Generated_Definition:
        new = word_map.get(defi, defi)
        new_defs.append(defi)


In [ ]:
dev = pd.read_csv('ru-dev-aya.csv', sep='\t', quoting=csv.QUOTE_NONE)


In [109]:
ari = adjusted_rand_score(train_new.sense_id, new_defs)
ari

0.11673766924128653